<a href="https://colab.research.google.com/github/cwattsnogueira/rating-predictor-spam-detection-review-summarizer/blob/main/15_02_review_summarization_bert_base_uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q nltk transformers gradio pandas

In [2]:
import os
import pandas as pd
import torch
import nltk

# Secure NLTK data folder (Colab /root/nltk_data is default)
NLTK_DATA_DIR = "/root/nltk_data"
os.makedirs(NLTK_DATA_DIR, exist_ok=True)

# Adds to NLTK path (ensures folder is searched)
if NLTK_DATA_DIR not in nltk.data.path:
    nltk.data.path.append(NLTK_DATA_DIR)

# Explicitly download 'punkt' into the above directory (show progress)
nltk.download('punkt', download_dir=NLTK_DATA_DIR)

# Optional debug: see where NLTK is looking for resources
print("nltk.data.path:", nltk.data.path)
try:
    # check if the file exists
    nltk.data.find('tokenizers/punkt/english.pickle')
    print("Punkt Available")
except LookupError:
    print("Punkt NOT found - fallback will be used")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


nltk.data.path: ['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']
Punkt Available


In [3]:
from nltk.tokenize import sent_tokenize

In [4]:
# Load BERT model
from transformers import BertTokenizer, BertModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.to(device)
model.eval()  # evaluation mode


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [5]:
import re
import traceback

def clean_reviews(text_list):
    # accepts both single string and list
    if isinstance(text_list, str):
        text_list = [text_list]
    cleaned = [str(t).strip().replace("\n", " ") for t in text_list if isinstance(t, str) and t.strip()]
    return cleaned

def split_to_sentences(text):
    """Tenta sent_tokenize; se falhar, usa fallback regex."""
    try:
        # force English explicitly
        sentences = sent_tokenize(text, language='english')
        # filter empty
        return [s.strip() for s in sentences if s.strip()]
    except Exception:
        # simple fallback (not perfect, but avoids crash)
        parts = re.split(r'(?<=[.!?])\s+', text)
        return [p.strip() for p in parts if p.strip()]

def get_sentence_embeddings(sentences, batch_size=32):
    """Retorna tensor [num_sentences, hidden_size] (em CPU)."""
    all_embeds = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            # CLS vector (position 0)
            cls_vecs = outputs.last_hidden_state[:, 0, :].cpu()
            all_embeds.append(cls_vecs)
    if len(all_embeds) == 0:
        return torch.empty((0, model.config.hidden_size))
    return torch.cat(all_embeds, dim=0)

In [6]:
def extractive_summary(product_name, reviews, top_k=3, preserve_order=True):
    try:
        reviews = clean_reviews(reviews)
        if not reviews:
            return "No valid reviews provided."

        text = " ".join(reviews)
        sentences = split_to_sentences(text)
        if not sentences:
            return "No valid sentences found."

        # Security: limit number of processed sentences (optional)
        MAX_SENTENCES = 2000
        if len(sentences) > MAX_SENTENCES:
            sentences = sentences[:MAX_SENTENCES]

        embeddings = get_sentence_embeddings(sentences)
        if embeddings.shape[0] == 0:
            return "No embeddings could be computed."

        # score = L2 norm of the CLS vector (simple, fast)
        scores = embeddings.norm(p=2, dim=1)
        k = min(top_k, len(sentences))
        topk_indices = torch.topk(scores, k).indices.tolist()

        if preserve_order:
            topk_indices = sorted(topk_indices)  # maintains original text order

        selected = [sentences[i] for i in topk_indices]
        summary_md = f"**Product:** {product_name}\n\n**Extractive Summary:**\n\n" + " ".join(selected)
        return summary_md
    except Exception as e:
        return "Error during summary generation: " + str(e) + "\n\n" + traceback.format_exc()


In [7]:
def summarize_from_csv(product_name, file, column_name="review_text"):
    try:
        # In gradio, file is an object with .name pointing to the temporary local path
        df = pd.read_csv(file.name)
        if column_name not in df.columns:
            return f"Column '{column_name}' not found in uploaded CSV. Columns available: {list(df.columns)}"
        reviews = df[column_name].astype(str).tolist()
        return extractive_summary(product_name, reviews)
    except Exception as e:
        return f"Error processing CSV: {e}\n\n" + traceback.format_exc()

In [8]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("##  Review Extractive Summarization App (BERT)")
    gr.Markdown("Upload a CSV or paste reviews manually. Include the product name to personalize the summary.")

    with gr.Tab("Manual Input"):
        product_name_manual = gr.Textbox(label="Product Name", placeholder="e.g., Wireless Headset")
        manual_input = gr.Textbox(lines=12, placeholder="Paste multiple reviews here (one per line)...")
        manual_button = gr.Button("Summarize")
        manual_output = gr.Markdown()  # show markdown

    with gr.Tab("CSV Upload"):
        product_name_csv = gr.Textbox(label="Product Name", placeholder="e.g., Wireless Headset")
        csv_input = gr.File(label="Upload CSV")
        column_name = gr.Textbox(value="review_text", label="Column name with reviews")
        csv_button = gr.Button("Summarize CSV")
        csv_output = gr.Markdown()

    def manual_click(name, text):
        # Split by rows and filter out empty ones
        lines = [l for l in text.split("\n") if l.strip()]
        return extractive_summary(name or "Unknown Product", lines)

    manual_button.click(
        manual_click,
        inputs=[product_name_manual, manual_input],
        outputs=manual_output
    )

    csv_button.click(
        summarize_from_csv,
        inputs=[product_name_csv, csv_input, column_name],
        outputs=csv_output
    )

demo.launch(share=False)  # in Colab use share=True if you want a public link

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>